# Tentative Table of content:
#### 1. Goal of the tutorial
#### 2. What is RSA! What could we use it for?
#### 3. How to build an RDM?
#### 4. Getting real: loading a brain dataset (fMRI or MEG ?)
#### 5. Prepare the data in the format we need
#### 6. Build and visualise the RDM (for different brain regions)
#### 7. Getting a pretrained model on Object Recognition!
#### 8. Extracting Activations! Building RDMs
#### 9. Computing RSA and visualizing results
#### 10. What about other modalities ! M/EEG ?
#### 11. Noise Ceiling and Untrained models!

#### List of exercies
